<style>
      h1, h2, h3, h4 {
        color: #bbb;
      }
      ol {
        counter-reset: item;
        font-size: 1.2rem;
        font-weight: 200;
      }
      li {
        display: block;
        color: #aaa;
      }
      li:before {
        content: counters(item, ".") " ";
        counter-increment: item;
      }
    </style>

<h1>Análise de Endereço dos Estabelecimentos</h1>
<ol>
    <li>importar as depedências</li>
    <li>configurar as variáveis de ambiente</li>
    <li>fazer download da base de dados
      <ol>
          <li>fazer a leitura da base com pandas</li>
      </ol>
    </li>
    <li>fazer a exploração da base
      <ol>
          <li>implementar lógica de exclusão de dados nulos</li>
          <li>fazer atualização da base de cep</li>
          <li>fazer atualização da base de estabelecimento usando novos ceps</li>
          <li>implementar novo modelo de dados estruturado para salvar</li>
      </ol>
    </li>
    <li>criar a rotina de tratamento dos dados</li>
</ol>


<h3 style="color: #ccc">1. Importar as depedências de desenvolvimento</h3>

In [2]:
import os, sys
import pandas as pd
import json
import csv
from unicodedata import normalize
from urllib import request
from datetime import datetime

sys.path.append(os.path.abspath(os.path.join('..')))

In [3]:
from services.version_service import VersionService
from keys.generate_key import GenerateKey as key
ver_service = VersionService()

In [4]:
STORAGE = os.path.join('..', 'storage')

In [5]:
ver_service.get_last_version('cep',key.EXT_JSON, STORAGE)

'cep.1.0.1.json'

In [6]:
FILENAME_TB_ESTABELECIMENTO = ver_service.get_last_version(key.FILENAME_TB_ESTABELECIMENTO, key.EXT_CSV, STORAGE)
FILENAME_CEP = ver_service.get_last_version(key.FILENAME_CEP, key.EXT_CSV, STORAGE)
FILENAME_JSON = ver_service.get_last_version(key.FILENAME_CEP,key.EXT_JSON, STORAGE)

<h3 style="color: #ccc">2. Configurar as variáveis de ambiente</h3>

In [7]:
environment = {}

with open(os.path.join('../', '.env'), encoding='utf-8') as env:
    files = env.readlines()
    
    for file in files:
        environment[str(file.split('=')[0])] = str(file.split('=')[1]).strip()
    env.close()

environment

{'URL': 'http://localhost', 'PORT': '8080'}

In [8]:
def get_url(path: str) -> str:
    return '{}:{}{}'.format(environment.get('URL'), environment.get('PORT'), path)

<h3 style="color: #ccc">3. Fazer download da base de dados</h3>

In [ ]:
# WARNING: run if not exist file
response = request.urlopen(get_url('/export-csv'))
raw_data = response.read()
encoding = response.info().get_content_charset('utf8')
filename = ver_service.get_next_version(key.FILENAME_TB_ESTABELECIMENTO, key.EXT_CSV, STORAGE)

with open(os.path.join(STORAGE, filename), 'w', encoding='utf-8') as csv:
    csv.writelines(raw_data.decode(encoding))
    csv.close()

<h4 style="color: #ccc">3.1 fazer a leitura da base com pandas</h4>

In [9]:
# from server
# df = pd.read_csv(get_url('/export-csv'), sep=';', encoding='utf-8', low_memory=False, dtype=str)

# from local
df = pd.read_csv(os.path.join(STORAGE, FILENAME_TB_ESTABELECIMENTO), sep=';', encoding='utf-8', low_memory=False, dtype=str)
df_cep = pd.read_csv(os.path.join(STORAGE, FILENAME_CEP), sep=',', encoding='utf-8', low_memory=False, dtype=str)

<h3 style="color: #ccc">4. fazer a exploração da base</h3>

In [11]:
# total de registro e colunas
print('estabelecimento: {}'.format(df.shape))
print('cep: {}'.format(df_cep.shape))

estabelecimento: (7538, 28)
cep: (732763, 5)


In [52]:
# detalhes sobre o dataframe
print(df.info())
print(80*'#')
print(df_cep.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7538 entries, 0 to 7537
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ATIVIDADES_SECUNDARIA    0 non-null      object
 1   ATIVIDADE_PRINCIPAL      7538 non-null   object
 2   BAIRRO                   7454 non-null   object
 3   CEP                      7524 non-null   object
 4   CNPJ                     7538 non-null   object
 5   CNPJ_DIGITO_VERIFICADOR  7538 non-null   object
 6   CNPJ_ORDEM               7538 non-null   object
 7   CNPJ_RAIZ                7538 non-null   object
 8   COMPLEMENTO              2661 non-null   object
 9   DATA_INICIO_ATIVIDADE    7538 non-null   object
 10  DATA_SITUACAO_CADASTRAL  7538 non-null   object
 11  DATA_SITUACAO_ESPECIAL   0 non-null      object
 12  DDD1                     5356 non-null   object
 13  DDD2                     571 non-null    object
 14  DDD_FAX                  545 non-null   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732763 entries, 0 to 732762
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   cep            732763 non-null  object
 1   cidade_estado  732763 non-null  object
 2   bairro         720745 non-null  object
 3   lougradouro    722723 non-null  object
 4   complemento    26309 non-null   object
dtypes: object(5)
memory usage: 14.0+ MB


None

In [15]:
# set max columns display
# show only five first registers
pd.set_option('display.max_columns', None)

In [16]:
display(df.head())

,ATIVIDADES_SECUNDARIA,ATIVIDADE_PRINCIPAL,BAIRRO,CEP,CNPJ,CNPJ_DIGITO_VERIFICADOR,CNPJ_ORDEM,CNPJ_RAIZ,COMPLEMENTO,DATA_INICIO_ATIVIDADE,DATA_SITUACAO_CADASTRAL,DATA_SITUACAO_ESPECIAL,DDD1,DDD2,DDD_FAX,EMAIL,ESTADO,FAX,ID,LOGRADOURO,NOME_FANTASIA,NUMERO,SITUACAO_CADASTRAL,SITUACAO_ESPECIAL,TELEFONE1,TELEFONE2,TIPO,TIPO_LOGRADOURO
0,NaN,8532500,UNIVERSITARIO,58429900,05055128001490,90,0014,05055128,SETOR C BLOCO BQ,20020410,20020410,NaN,83,83,21011625,diretor@ctrn.ufcg.edu.br,PB,21011400,13,APRIGIO VELOSO,CENTRO DE TECNOLOGIA E RECURSOS NATURAIS DA UFCG,882,02,NaN,21011400,21011625,2,RUA
1,NaN,8299799,CENTRO,69900088,05022353003393,93,0033,05022353,SALA 46 EDIF GALERIA CUNHA,20140904,20170220,NaN,68,NaN,NaN,ESCGADELHA@BOL.COM.BR,AC,32243750,12,CEARA,RV TECNOLOGIA E SISTEMAS,1221,08,NaN,32230092,NaN,2,AVENIDA
2,NaN,4649499,IPIRANGA,04208002,05095076000242,42,0002,05095076,LOJA 2598,20140911,20211217,NaN,11,NaN,NaN,MARCODONASCIMENTO@HOTMAIL.COM.BR,SP,NaN,14,SILVA BUENO,NaN,2576,08,NaN,20682945,NaN,2,RUA
3,NaN,8660700,CENTRO,15110000,00954916000261,61,0002,00954916,NaN,20140725,20140725,NaN,11,NaN,11209748,SSRN@UOL.COM.BR,SP,NaN,1,ABRAHAO JOSE DE LIMA,GOSSEN,512,02,NaN,20974848,11209748,2,AVENIDA
4,NaN,4731800,VITORIA DA CONQUISTA,46101000,05291198000204,04,0002,05291198,NaN,20140905,20140905,NaN,77,NaN,NaN,POSTOMEIRA01BDO@HOTMAIL.COM,BA,34413328,15,BR 030,POSTO MEIRA II,286,02,NaN,34417851,NaN,2,RODOVIA


In [17]:
df_address = df[['ID','CEP', 'BAIRRO', 'COMPLEMENTO', 'ESTADO', 'LOGRADOURO', 'NUMERO', 'TIPO_LOGRADOURO']]

In [55]:
display(df_address.head())
print(80*'#')
display(df_cep.head())

,ID,CEP,BAIRRO,COMPLEMENTO,ESTADO,LOGRADOURO,NUMERO,TIPO_LOGRADOURO
0,13,58429900,UNIVERSITARIO,SETOR C BLOCO BQ,PB,APRIGIO VELOSO,882,RUA
1,12,69900088,CENTRO,SALA 46 EDIF GALERIA CUNHA,AC,CEARA,1221,AVENIDA
2,14,04208002,IPIRANGA,LOJA 2598,SP,SILVA BUENO,2576,RUA
3,1,15110000,CENTRO,NaN,SP,ABRAHAO JOSE DE LIMA,512,AVENIDA
4,15,46101000,VITORIA DA CONQUISTA,NaN,BA,BR 030,286,RODOVIA


################################################################################


,cep,cidade_estado,bairro,lougradouro,complemento
0,01001000,São Paulo/SP,Sé,Praça da Sé - lado ímpar,NaN
1,01001001,São Paulo/SP,Sé,Praça da Sé - lado par,NaN
2,01001010,São Paulo/SP,Sé,Rua Filipe de Oliveira,NaN
3,01001900,São Paulo/SP,Sé,"Praça da Sé, 108",UNESP - Universidade Estadual Júlio de Mesquit...
4,01001901,São Paulo/SP,Sé,"Praça da Sé, 371",Edifício Santa Lídia


<h3 style="color: #ccc">4.1 implementar lógica de exclusão de dados nulos</h3>

In [20]:
# check if exits column null and sum them
display(df.isnull().sum())

ATIVIDADES_SECUNDARIA      7538
ATIVIDADE_PRINCIPAL           0
BAIRRO                       84
CEP                          14
CNPJ                          0
CNPJ_DIGITO_VERIFICADOR       0
CNPJ_ORDEM                    0
CNPJ_RAIZ                     0
COMPLEMENTO                4877
DATA_INICIO_ATIVIDADE         0
DATA_SITUACAO_CADASTRAL       0
DATA_SITUACAO_ESPECIAL     7538
DDD1                       2182
DDD2                       6967
DDD_FAX                    6993
EMAIL                      4085
ESTADO                        1
FAX                        6748
ID                            0
LOGRADOURO                    0
NOME_FANTASIA              2657
NUMERO                        0
SITUACAO_CADASTRAL            0
SITUACAO_ESPECIAL          7507
TELEFONE1                  2181
TELEFONE2                  6993
TIPO                          0
TIPO_LOGRADOURO             171
dtype: int64

In [41]:
display(df_cep.isnull().sum())
print(80*"#")
display(df_address.isnull().sum())

cep                   0
cidade_estado         0
bairro            12018
lougradouro       10040
complemento      706454
dtype: int64

################################################################################


ID                    0
CEP                  14
BAIRRO               84
COMPLEMENTO        4877
ESTADO                1
LOGRADOURO            0
NUMERO                0
TIPO_LOGRADOURO     171
dtype: int64

In [32]:
# count occurence
display(df_address.value_counts())
print(80*'#')
display(df_cep.value_counts())

ID    CEP       BAIRRO                  COMPLEMENTO  ESTADO  LOGRADOURO             NUMERO  TIPO_LOGRADOURO
1000  83750000  Centro                  ANEXO        PR      Doutor Manoel Pedro    2213    AVENIDA            1
5719  03924040  JARDIM DONA SINHA       A            SP      NOVA BRASILIA          622     RUA                1
5700  75040010  CENTRO                  A            GO      TIRADENTES             453     AVENIDA            1
5702  86020041  CENTRO                  SALA 07      PR      SANTOS                 1225    RUA                1
5705  72850615  JARDIM ZULEIKA          QD 04 LT 20  GO      CACAPAVA               S/N     RUA                1
                                                                                                              ..
3429  11703100  TUPI                    CASA 03      SP      JOSE DA SILVA MACHADO  512     RUA                1
3436  02517000  JARDIM DAS LARANJEIRAS  LOJA 8 E 9   SP      RELIQUIA               383     RUA      

################################################################################


cep       cidade_estado       bairro                                  lougradouro                                complemento                                          
01001900  São Paulo/SP        Sé                                      Praça da Sé, 108                           UNESP - Universidade Estadual Júlio de Mesquita Filho    1
64020900  Teresina/PI         Parque São João                         Rua Jacob Martins, s/n                     Condomínio Verde Que Te Quero Verde                      1
64045903  Teresina/PI         São João                                Rua Padre José Rêgo, 748                   Edifício San Michael                                     1
64045902  Teresina/PI         Noivos                                  Rua Padre Cirilo Chaves, 1877              Condomínio Vinícius de Moraes                            1
64045901  Teresina/PI         São João                                Avenida Noronha Almeida, 2050              Condomínio Vila Lobos           

In [45]:
# find id duplicated
display('CNPJ duplicated: {}'.format(df[df.duplicated('CNPJ')]['ID'].values))

'CNPJ duplicated: []'

In [42]:
display(df_address['CEP'].isin(df_cep['cep']).value_counts())
print(80*'#')
display(df_cep['cep'].isin(df['CEP']).value_counts())

CEP
True     6253
False    1285
Name: count, dtype: int64

################################################################################


cep
False    727577
True       5186
Name: count, dtype: int64

In [45]:
merge = pd.merge(df_address, df_cep, left_on='CEP', right_on='cep', how='inner')

In [46]:
merge.drop('cep', axis=1)

,ID,CEP,BAIRRO,COMPLEMENTO,ESTADO,LOGRADOURO,NUMERO,TIPO_LOGRADOURO,cidade_estado,bairro,lougradouro,complemento
0,13,58429900,UNIVERSITARIO,SETOR C BLOCO BQ,PB,APRIGIO VELOSO,882,RUA,Campina Grande/PB,Universitário,"Rua Aprígio Veloso, 882",Universidade Federal de Campina Grande
1,17,79700000,CENTRO EDUCACIONAL,NaN,MS,EMILIANA BARROS,1005,RUA,Fátima do Sul/MS,NaN,NaN,NaN
2,12,69900088,CENTRO,SALA 46 EDIF GALERIA CUNHA,AC,CEARA,1221,AVENIDA,Rio Branco/AC,Centro,Avenida Ceará - de 957 a 1857 - lado ímpar,NaN
3,18,96204036,ZONA PORTUARIA,NaN,RS,ANTONIO JOAO BIANCHINI,940,AVENIDA,Rio Grande/RS,Zona Portuária,Avenida Antônio João Bianchini,NaN
4,19,85935000,JARDIM PARANA,SALA 01,PR,TUPASSI,486,AVENIDA,Assis Chateaubriand/PR,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6249,7727,98865000,NaN,CASA,RS,PINTO BANDEIRA,SN,RUA,São Miguel das Missões/RS,NaN,NaN,NaN
6250,7729,90040140,SANTANA,03,RS,LAURINDO,228,RUA,Porto Alegre/RS,Santana,Rua Laurindo,NaN
6251,7732,97110270,CAMOBI,NaN,RS,EQUADOR,126,RUA,Santa Maria/RS,Camobi,Rua Equador,NaN
6252,7734,90050230,CENTRO,NaN,RS,JOAO ALFREDO,387,RUA,Porto Alegre/RS,Cidade Baixa,Rua João Alfredo,NaN


In [49]:
# def renameColumn(df: pd.DataFrame, value: dict) -> None:
#     df.rename(columns=value, inplace=True)


# def join_doc_company_number(df: pd.DataFrame) -> None:
#     join1 = lambda v1, v2: int(str(v1) + '000' + str(v2))
#     join2 = lambda v1, v2: int(str(v1) + '0' + str(v2)) if len(str(v2)) == 1 else int(str(v1) + str(v2))
#     df['DOC_COMPANY_NUMBER'] = df['COL0'].combine(df['COL1'], join1).combine(df['COL2'], join2)


# def join_address(df: pd.DataFrame, cols: list) -> None:
#     join = lambda s1, s2: s1 + ' ' + s2
    
#     for col in df[cols]:
#         df['ADDRESS_COMPLETE'] = df['BAIRRO'].combine(df[col], join)


# def to_lower(col: pd.Series) -> None:
#     col.fillna(value='', inplace=True)
#     col.str.lower()


# def to_int(col: pd.Series) -> pd.Series:
#     return col.apply(np.int64)


# def to_date(col: pd.Series) -> pd.Series:
#     return pd.to_datetime(col, format="%Y%m%d")

# normalize unicode
def normalize_unicode(text: str) -> str:
    return normalize('NFKD', text).encode('ASCII','ignore').decode('ASCII')

# jon column and remove duplicated
def join_columns(text: str) -> str:
    return ' '.join(dict.fromkeys(normalize_unicode(text).lower().split()))

# combine column
def combine_columns(df: pd.DataFrame, col_source: str, col_target: str, delimiter:str = ' '):
    join = lambda s1, s2: join_columns(str(s1 + delimiter + s2))
    df[col_source] = df[col_source].combine(df[col_target], join)

# fillna columns
def filla_columns(df: pd.DataFrame, cols: list, value = ''):
    for col in cols:
        df[col].fillna(value=value, inplace=True)

def to_lower(df: pd.DataFrame, col: str):
    df[col] = df[col].str.lower()

def csv_to_json():
    jsonArray = []
      
    #read csv file
    with open(os.path.join(STORAGE, FILENAME_CEP), 'r', encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
        csvf.close()
    
    #convert python jsonArray to JSON String and write to file
    write_to = os.path.join(STORAGE, ver_service.get_next_version(key.FILENAME_CEP, key.EXT_JSON, STORAGE))
    with open(write_to, 'w', encoding='utf-8') as jsonf:
        json.dump(jsonArray, jsonf, ensure_ascii=False, indent=4)
        jsonf.close()

In [47]:
to_lower(merge, 'TIPO_LOGRADOURO')
to_lower(merge, 'BAIRRO')

In [48]:
filla_columns(merge, ['bairro', 'complemento', 'lougradouro', 'COMPLEMENTO', 'BAIRRO', 'TIPO_LOGRADOURO'])

In [49]:
combine_columns(merge, 'COMPLEMENTO', 'complemento')
combine_columns(merge, 'LOGRADOURO', 'lougradouro')
combine_columns(merge, 'LOGRADOURO', 'TIPO_LOGRADOURO')


In [52]:
merge['CIDADE'] = merge['cidade_estado'].map(lambda x: normalize_unicode(str(x.split('/')[0]).lower()))

In [ ]:
merge.rename(columns={'ID': 'ESTABELECIMENTO_ID'}, inplace=True)

In [61]:
merge[merge.columns.difference(['cidade_estado', 'cep', 'bairro', 'complemento', 'lougradouro'])]

,BAIRRO,CEP,CIDADE,COMPLEMENTO,ESTABELECIMENTO_ID,ESTADO,LOGRADOURO,NUMERO,TIPO_LOGRADOURO
0,universitario,58429900,campina grande,setor c bloco bq universidade federal de campi...,13,PB,"aprigio veloso rua veloso, 882",882,rua
1,centro educacional,79700000,fatima do sul,,17,MS,emiliana barros rua,1005,rua
2,centro,69900088,rio branco,sala 46 edif galeria cunha,12,AC,ceara avenida - de 957 a 1857 lado impar,1221,avenida
3,zona portuaria,96204036,rio grande,,18,RS,antonio joao bianchini avenida,940,avenida
4,jardim parana,85935000,assis chateaubriand,sala 01,19,PR,tupassi avenida,486,avenida
...,...,...,...,...,...,...,...,...,...
6249,,98865000,sao miguel das missoes,casa,7727,RS,pinto bandeira rua,SN,rua
6250,santana,90040140,porto alegre,03,7729,RS,laurindo rua,228,rua
6251,camobi,97110270,santa maria,,7732,RS,equador rua,126,rua
6252,centro,90050230,porto alegre,,7734,RS,joao alfredo rua,387,rua


In [167]:
cep_left = df[~df['CEP'].isin(df_cep['cep'])]

In [168]:
cep_left = cep_left[['CEP', 'BAIRRO', 'COMPLEMENTO', 'ESTADO', 'LOGRADOURO', 'NUMERO']]

In [169]:
cep_left.rename(columns={'CEP':'cep','BAIRRO':'bairro','COMPLEMENTO':'complemento','LOGRADOURO':'lougradouro','NUMERO':'numero','ESTADO':'estado'}, inplace=True)

In [170]:
cep_left['cidade_estado'] = cep_left['bairro']

In [171]:
cep_left

,cep,bairro,complemento,estado,lougradouro,numero,cidade_estado
4,46101000,VITORIA DA CONQUISTA,NaN,BA,BR 030,286,VITORIA DA CONQUISTA
11,57071403,CLIMA BOM 1,NaN,AL,JERUSALEM,60,CLIMA BOM 1
13,14300001,CENTRO,NaN,SP,CONEGO JOAQUIM ALVES,195,CENTRO
14,34000001,JARDIM CANADA,NaN,MG,OLIVER,144,JARDIM CANADA
28,95700166,CENTRO,LOJA 2,RS,BARAO DO RIO BRANCO,402,CENTRO
...,...,...,...,...,...,...,...
7502,99700010,FLORESTINHA,NaN,RS,SIDINEI GUERRA,375,FLORESTINHA
7506,99700010,CENTRO,NaN,RS,RAUL MIRANDES SILVA,44,CENTRO
7524,99700010,BELA VISTA,NaN,RS,ERNESTO RIGONI,299,BELA VISTA
7533,95600000,TUCANOS,NaN,RS,28 DE JULHO,2481,TUCANOS


In [156]:
cep_left.isnull().sum()

cep               14
bairro            38
complemento      854
estado             1
lougradouro        0
numero             0
cidade_estado      0
dtype: int64

In [144]:
len(cep_left['cep'].iloc[0])

8

In [172]:
filla_columns(cep_left, ['estado', 'bairro', 'complemento', 'cidade_estado'])
filla_columns(cep_left, ['cep'], '00000000')

In [173]:
combine_columns(cep_left, 'cidade_estado', 'estado', '/')

In [174]:
combine_columns(cep_left, 'lougradouro', 'numero', ', ')

In [175]:
cep_left.isnull().sum()

cep              0
bairro           0
complemento      0
estado           0
lougradouro      0
numero           0
cidade_estado    0
dtype: int64

In [176]:
cep_left

,cep,bairro,complemento,estado,lougradouro,numero,cidade_estado
4,46101000,VITORIA DA CONQUISTA,,BA,"br 030, 286",286,vitoria da conquista/ba
11,57071403,CLIMA BOM 1,,AL,"jerusalem, 60",60,clima bom 1/al
13,14300001,CENTRO,,SP,"conego joaquim alves, 195",195,centro/sp
14,34000001,JARDIM CANADA,,MG,"oliver, 144",144,jardim canada/mg
28,95700166,CENTRO,LOJA 2,RS,"barao do rio branco, 402",402,centro/rs
...,...,...,...,...,...,...,...
7502,99700010,FLORESTINHA,,RS,"sidinei guerra, 375",375,florestinha/rs
7506,99700010,CENTRO,,RS,"raul mirandes silva, 44",44,centro/rs
7524,99700010,BELA VISTA,,RS,"ernesto rigoni, 299",299,bela vista/rs
7533,95600000,TUCANOS,,RS,"28 de julho, 2481",2481,tucanos/rs


In [136]:
# cep_left.rename(columns={'bairro':'cidade_estado'}, inplace=True)

In [177]:
cep_left[cep_left.columns.difference(['estado', 'numero'])]

,bairro,cep,cidade_estado,complemento,lougradouro
4,VITORIA DA CONQUISTA,46101000,vitoria da conquista/ba,,"br 030, 286"
11,CLIMA BOM 1,57071403,clima bom 1/al,,"jerusalem, 60"
13,CENTRO,14300001,centro/sp,,"conego joaquim alves, 195"
14,JARDIM CANADA,34000001,jardim canada/mg,,"oliver, 144"
28,CENTRO,95700166,centro/rs,LOJA 2,"barao do rio branco, 402"
...,...,...,...,...,...
7502,FLORESTINHA,99700010,florestinha/rs,,"sidinei guerra, 375"
7506,CENTRO,99700010,centro/rs,,"raul mirandes silva, 44"
7524,BELA VISTA,99700010,bela vista/rs,,"ernesto rigoni, 299"
7533,TUCANOS,95600000,tucanos/rs,,"28 de julho, 2481"


In [71]:
df_cep

,cep,cidade_estado,bairro,lougradouro,complemento
0,01001000,São Paulo/SP,Sé,Praça da Sé - lado ímpar,NaN
1,01001001,São Paulo/SP,Sé,Praça da Sé - lado par,NaN
2,01001010,São Paulo/SP,Sé,Rua Filipe de Oliveira,NaN
3,01001900,São Paulo/SP,Sé,"Praça da Sé, 108",UNESP - Universidade Estadual Júlio de Mesquit...
4,01001901,São Paulo/SP,Sé,"Praça da Sé, 371",Edifício Santa Lídia
...,...,...,...,...,...
732758,99975970,São João Bosco (Ciríaco)/RS - Distrito,Centro,"Rua Principal, s/n",AGC São João Bosco
732759,99978000,Cruzaltinha (Ciríaco)/RS - Distrito,NaN,NaN,NaN
732760,99980000,David Canabarro/RS,NaN,NaN,NaN
732761,99980970,David Canabarro/RS,Centro,"Rua Adelino Gazzoni, 160",AC David Canabarro


In [179]:
df_cep_updated = pd.concat([df_cep, cep_left])

In [180]:
df_cep_updated

,cep,cidade_estado,bairro,lougradouro,complemento,estado,numero
0,01001000,São Paulo/SP,Sé,Praça da Sé - lado ímpar,NaN,NaN,NaN
1,01001001,São Paulo/SP,Sé,Praça da Sé - lado par,NaN,NaN,NaN
2,01001010,São Paulo/SP,Sé,Rua Filipe de Oliveira,NaN,NaN,NaN
3,01001900,São Paulo/SP,Sé,"Praça da Sé, 108",UNESP - Universidade Estadual Júlio de Mesquit...,NaN,NaN
4,01001901,São Paulo/SP,Sé,"Praça da Sé, 371",Edifício Santa Lídia,NaN,NaN
...,...,...,...,...,...,...,...
7502,99700010,florestinha/rs,FLORESTINHA,"sidinei guerra, 375",,RS,375
7506,99700010,centro/rs,CENTRO,"raul mirandes silva, 44",,RS,44
7524,99700010,bela vista/rs,BELA VISTA,"ernesto rigoni, 299",,RS,299
7533,95600000,tucanos/rs,TUCANOS,"28 de julho, 2481",,RS,2481


In [181]:
df_cep_updated.shape[0] - df_cep.shape[0]

1285

In [ ]:
# renameColumn(df_copy, {'COL4':'COMPANY_NAME'})

In [ ]:
# df_filtered = df_copy[df_copy.columns.difference([])]

In [ ]:
# df_filtered['ZIP_CODE'].fillna(value=0, inplace=True)
# df_filtered['ZIP_CODE'] = to_int(df_filtered['ZIP_CODE'])

In [ ]:
# df_filtered['DATE_CLOSED'] = to_date(df_filtered['DATE_CLOSED'])
# df_filtered['DATE_OPENED'] = to_date(df_filtered['DATE_OPENED'])

In [ ]:
# description_category.loc[description_category['listed_in'] = description_category['listed_in'].apply(lambda x: x.split(', '))]
# df_filtered['STREET_COMPLEMENT'] = df_filtered['STREET_COMPLEMENT'].str.replace(';', '')
# df_filtered['STREET_COMPLEMENT'] = df_filtered['STREET_COMPLEMENT'].str.replace(':', ' ')

In [ ]:
# df_filtered['DATE_EN'] = pd.to_datetime(df_filtered['DATE_EN'], format="%Y%m%d")
# df_filtered['DATE_ST'] = pd.to_datetime(df_filtered['DATE_ST'], format="%Y%m%d")

In [ ]:
# df_filtered[(df_filtered['DATE_EN'] >= '2023-01-01') & ( df_filtered['STATE'] == 'SP')]

In [ ]:
# response = request.urlopen('https://publica.cnpj.ws/cnpj/20857537000180')
# raw_data = response.read()
# encoding = response.info().get_content_charset('utf8')  # JSON default
# data = json.loads(raw_data.decode(encoding))

In [ ]:
# df['EMAIL'] = df['EMAIL'].str.lower()

# set(df_address['CEP']).intersection(set(df_cep['cep']))

# json para envio ao backend 
# bytes(str(json.dumps(df.iloc[0].to_dict())).lower(), encoding='utf-8')